In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import time
import ssl
import os

In [3]:
#For ignoring SSL ceritficate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [4]:
#helper function get first number in string

def str_first_num(s):
    first_num = ''
    flag_digit=False
    for el in s: 
        if el.isdigit():
            first_num += el
            flag_digit = True
        else:
            if flag_digit:
                break
    return first_num
    

In [5]:
# a helper function that removes unwanted words/sections like url links
def strip_rgx_words(inp, regex_ = ['image','url']):
    #remove urls or images
    rgx_word = None
    for rgx in regex_:
      if rgx in inp:
          rgx_word = rgx
          break
    if rgx_word is not None:
      out = inp.split(rgx_word)[0]
    else:
      out =inp
    return out

In [6]:
#read question urls from csv file
src_path = r"C:\Users\Ravit\Documents\horizon_scanning_lab\Scrapers\Quora_RCS\answers_and_info\quet_indexes.csv"
df = pd.read_csv(src_path)

In [7]:
#get list of urls
url_list = list(df['url'].values)

In [8]:
url_list

['https://www.quora.com/What-will-the-worlds-technology-be-like-in-50-years',
 'https://www.quora.com/How-long-could-the-brain-survive-theoretically-if-we-had-the-technology-to-replace-all-other-body-parts-as-you-aged-with-functioning-organs-that-are-grown-in-a-lab-using-your-DNA',
 'https://www.quora.com/What-are-the-top-10-emerging-technologies-in-the-next-5-10-years-2020%E2%80%932025',
 'https://www.quora.com/What-are-the-upcoming-emerging-technologies-in-software-industry',
 'https://www.quora.com/What-will-the-worlds-technology-be-like-in-50-years',
 'https://www.quora.com/What-are-the-most-advanced-technologies-that-people-dont-know-about-yet',
 'https://www.quora.com/When-will-Fusion-reactors-become-a-reality',
 'https://www.quora.com/In-future-IOT-Internet-of-things-is-trending-technology-or-not',
 'https://www.quora.com/What-will-the-worlds-technology-be-like-in-50-years',
 'https://www.quora.com/What-are-the-solutions-to-emerging-issues-in-communication']

In [35]:
def answers_into_df(url):
    '''input: url for scrapping answers to a particular question.
        out: df with answers for each url/question'''
    
    #request url html
    page = requests.get(url)
   
    #create soup object with htmal parser
    soup = BeautifulSoup(page.content, 'html.parser')

    answers = soup.find_all('script', {"type": "application/ld+json"})[0]
    #convert soup object to string
    answers = [str(a) for a in answers][0]
    #split answers with text separator
    answers_list = answers.split('"text":')
    #first element is just the header of a page. remove it
    answers_list = answers_list[1:]
    
    #create df with nans to fill it later with values
    columns = ["Answer", "upvoteCount", "affiliation", "answerCount", "followerCount", "hasCredential",  "knowsAbout", "name"]
    x_shape = (len(answers_list), len(columns))
    x = np.tile(np.nan, x_shape)
   
    answers_df = pd.DataFrame(x, columns = columns)

    for i, ans in enumerate(answers_list):    
        #get list of categories that exist in answer post
        existing_columns = [col for col in columns if col in ans]
               
        #get values to columns that exist in post 
        regex_ = ["image", "url"]
        
        for j, c in enumerate(existing_columns):          
            #Fill first element column which is 'Answer'
            if c=='Answer':
                split_post = ans.split(c)
                answers_df[c].iloc[i] = split_post[0]
                split_post = split_post[1]
                continue   
            
            #if category is not the last in existing list         
            if c!=existing_columns[-1]:
                next_c = existing_columns[j+1]
                #get text relevant to c -column
                split_post = split_post.split(c)
                inter_split = split_post[-1].split(next_c)
                #insert regex helper function here----!
                #if category is numeric get number
                if "Count" in c:
                    answers_df[c].iloc[i] = str_first_num(inter_split[0])
                split_post = inter_split[-1]
                continue
            
            if "Count" in c:
                answers_df[c].iloc[i] = str_first_num(split_post)
                continue
            #insert regex helper function here----!
            answers_df[c].iloc[i] = split_post  
    return answers_df 
 

In [36]:
answers_into_df(url_list[0])

C:\Users\Ravit\AppData\Local\Temp\ipykernel_27864\1866742848.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answers_df[c].iloc[i] = str_first_num(split_post)
C:\Users\Ravit\AppData\Local\Temp\ipykernel_27864\1866742848.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answers_df[c].iloc[i] = str_first_num(inter_split[0])
C:\Users\Ravit\AppData\Local\Temp\ipykernel_27864\1866742848.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

,Answer,upvoteCount,affiliation,answerCount,followerCount,hasCredential,knowsAbout,name
0,"""50 years is a very long time. The way I make...",,NaN,NaN,NaN,NaN,NaN,NaN
1,"""Faster elevators capable of moving sideways....",,NaN,11177,815,NaN,NaN,""": ""T. Stephen Cornelius"", ""url"": ""https://www..."
2,"""INCOHERENT and Excessive redundancy as resou...",,NaN,12202,8904,NaN,NaN,""": ""Richard Kenneth Eng"", ""url"": ""https://www...."
3,"""Well, lucky for you, I just returned from 50...",,NaN,3659,181,NaN,NaN,""": ""Albert Hsieh"", ""url"": ""https://www.quora.c..."
4,"""1. 3D printing becomes the norm in hospitals...",,NaN,9824,21129,NaN,NaN,""": ""Stan Hayward"", ""url"": ""https://www.quora.c..."
5,"""Very easily\nThe technology in 50 years will...",,NaN,983,104,NaN,NaN,""": ""Steve Schaefer"", ""url"": ""https://www.quora..."
6,"""I believe we are fast approaching the peak o...",,NaN,1855,111,NaN,NaN,""": ""Anthony Musk"", ""url"": ""https://www.quora.c..."
7,"""Look at the videos from the 80s and 90s. Peo...",,NaN,2359,3099,NaN,NaN,""": ""EIS Intelligent Systems""}},"
8,"""1. Real Artificial Intelligence Takes Over.\...",,NaN,3871,968,NaN,NaN,""": ""Ernie Dunbar"", ""url"": ""https://www.quora.c..."
9,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#iterate through url_list and save multiple dfs
save_path = r"C:\Users\Ravit\Documents\horizon_scanning_lab\Scrapers\Quora_RCS\answers_and_info"
for i, url in enumerate(url_list):
    df = answers_into_df(url)
    final_path = os.path.join(save_path, "q_"+str(i) + ".csv")
    df.to_csv(final_path)

In [ ]:
### This section is for testing on a single url. Delete it after completing the crawler

In [ ]:
#request url html
page = requests.get(url_list[0])

#create soup object with htmal parser
soup = BeautifulSoup(page.content, 'html.parser')


In [ ]:
answers = soup.find_all('script', {"type": "application/ld+json"})[0]
#convert soup object to string
answers = [str(a) for a in answers][0]
#split answers with text separator
answers_list = answers.split('"text":')

In [ ]:
columns = ["Answer", "name", "knowsAbout",  "hasCredential", "upvoteCount", "affiliation", "answerCount", "followerCount"]

#create df with nans to fill it later with values
x_shape = (len(answers_list)-1, len(columns))
x = np.tile(np.nan, x_shape)
answers_df = pd.DataFrame(x, columns = columns)

In [ ]:
#get answer value
split_post = answers_list[1].split("upvoteCount")
answers_df["Answer"].iloc[0] = split_post[0]

#remove answer
split_post = split_post[1]

#get upvote value
upvote = str_first_num(split_post)

answers_df["upvoteCount"].iloc[0] = float(upvote)

#get affiliation which is right before answercount
split_post = split_post.split("affiliation")
#split_post = split_post[1]
split_post = split_post.split("answerCount")

answers_df["affiliation"].iloc[0] = split_post[0]

split_post = split_post[1]

#get answer count
answers_df["answerCount"].iloc[0] = str_first_num(split_post)

#get followercount
split_post = split_post.split("followerCount")
answers_df['followerCount'] .iloc[0]= str_first_num(split_post[1])

split_post = split_post[1]

#get person credentials (between hascredentials and knowsabout)
split_post = split_post.split("hasCredential")[1]
split_post = split_post.split("knowsAbout")

#remove url from credential part
answers_df['hasCredential'].iloc[0] = split_post[0].split("image")[0]
split_post = split_post[1]

#Finally, get knowsabout and name. Remove url image from knowsabout too
split_post = split_post.split("name")
answers_df['knowsAbout'].iloc[0] = split_post[0].split("image")[0]

answers_df['name'].iloc[0] = split_post[1]


 